Case 5: Multi-Period Analysis

This notebook will show the process of demand period configuration in the YAML settings file and illustrates the different output between different demand periods on different scale factors.

This process can be replicated by downloading this notebook along with the minimum input folder and running each line in sequence.

In [1]:
import pandas as pd
import ruamel.yaml as yaml
import subprocess
import matplotlib.pyplot as plt
from scipy.stats import linregress

Display the current demand periods and demand files

In [2]:
with open('settings.yml', 'r') as file:
    config = yaml.safe_load(file)

print(yaml.dump(config['demand_periods'], Dumper=yaml.RoundTripDumper))
print(yaml.dump(config['demand_files'], Dumper=yaml.RoundTripDumper))

- demand_period: am
  demand_period_id: 1
  time_period: '0700_0800'

- file_sequence_no: 1
  scenario_index_vector: 0
  file_name: demand.csv
  demand_period: am
  mode_type: auto
  format_type: column
  scale_factor: 1
  departure_time_profile_no: 1



Append a new demand period MD with a scale factor of 0.4 and the corresponding demand file, and display the same

In [3]:
x = {'demand_period': 'md', 'demand_period_id': 2, 'time_period': '1100_1200'}
y = {'file_sequence_no': 2,
     'scenario_index_vector': 0,
     'file_name': 'demand.csv',
     'demand_period': 'md',
     'mode_type': 'auto',
     'format_type': 'column',
     'scale_factor': 0.4,
     'departure_time_profile_no': 1}
if x not in config['demand_periods']:
    config['demand_periods'].append(x)
if y not in config['demand_files']:
    config['demand_files'].append(y)
            
with open('settings.yml', 'w') as file:
    yaml.dump(config, file, Dumper = yaml.RoundTripDumper)
    
print(yaml.dump(config['demand_periods'], Dumper=yaml.RoundTripDumper))
print(yaml.dump(config['demand_files'], Dumper=yaml.RoundTripDumper))

- demand_period: am
  demand_period_id: 1
  time_period: '0700_0800'
- demand_period: md
  demand_period_id: 2
  time_period: '1100_1200'

- file_sequence_no: 1
  scenario_index_vector: 0
  file_name: demand.csv
  demand_period: am
  mode_type: auto
  format_type: column
  scale_factor: 1
  departure_time_profile_no: 1
- file_sequence_no: 2
  scenario_index_vector: 0
  file_name: demand.csv
  demand_period: md
  mode_type: auto
  format_type: column
  scale_factor: 0.4
  departure_time_profile_no: 1



Run DTALite executable

In [4]:
command = 'DTALite'
try:
    subprocess.run(command, shell=True, check=True)
    print("DTALite executed successfully.")
except subprocess.CalledProcessError as e:
    print(f"Error executing DTALite: {e}")

DTALite executed successfully.


Total volume for each time period

In [5]:
df = pd.read_csv('link_performance.csv')
tot_vol = df.groupby('time_period')['vehicle_volume'].sum().reset_index()
tot_vol

,time_period,vehicle_volume
0,0700_0800,879272.813
1,1100_1200,345018.828


In [6]:
am_vol = tot_vol.loc[0]['vehicle_volume']
md_vol = tot_vol.loc[1]['vehicle_volume']
scale = md_vol/am_vol
print(f'The total volume for AM time period is {am_vol:1f}, and the total volume for MD time period is {md_vol:1f}')
print(f'The total volume for MD time period is {scale:1f} as much as AM time period, which is roughly 0.4')

The total volume for AM time period is 879272.813000, and the total volume for MD time period is 345018.828000
The total volume for MD time period is 0.392391 as much as AM time period, which is roughly 0.4


Average travel time for each time period

In [7]:
avg_tt = df.groupby('time_period')['travel_time'].mean().reset_index()
avg_fftt = df.groupby('time_period')['fftt'].mean().reset_index()
avg_tt_fftt = pd.merge(avg_tt, avg_fftt, on='time_period', suffixes=('_tt', '_fftt'))
avg_tt_fftt

,time_period,travel_time,fftt
0,0700_0800,8.788579,4.131579
1,1100_1200,4.405026,4.131579


Average speed (kmph) for each time period

In [8]:
df2 = pd.read_csv('link.csv')
avg_spd = df.groupby('time_period')['speed_kmph'].mean().reset_index()
avg_ffspd = df2['free_speed'].mean()
avg_spd['free_speed_kmph'] = avg_ffspd
avg_spd

,time_period,speed_kmph,free_speed_kmph
0,0700_0800,34.356763,60.0
1,1100_1200,56.045816,60.0
